In [186]:
import pandas as pd
import numpy as np

import bamboolib as bam

In [187]:
URL = 'https://raw.githubusercontent.com/globaldothealth/monkeypox/main/latest.csv'

data = pd.read_csv(URL)



In [188]:
data_pox =data[data.columns[data.isna().sum()/data.shape[0]<0.85]]
data_pox = data_pox.loc[~(data_pox['Status'].isin(['discarded']))]
data_pox = data_pox.drop(columns=['Location', 'Gender', 'Source_II', 'ID', 'Source','Date_last_modified'])

suspected_data_pox = data_pox.loc[data_pox['Status'].isin(['suspected'])]
suspected_data_pox['date'] = pd.to_datetime(suspected_data_pox['Date_entry'], format='%Y-%m-%d')
confirmed_data_pox = suspected_data_pox.groupby(['Country', 'date']).agg(suspected=('Status', 'size')).reset_index()


confirmed_data_pox = data_pox.loc[data_pox['Status'].isin(['confirmed'])]
confirmed_data_pox['date'] = pd.to_datetime(confirmed_data_pox['Date_confirmation'], format='%Y-%m-%d')
confirmed_data_pox = confirmed_data_pox.groupby(['Country', 'date']).agg(confirmed=('Status', 'size')).reset_index()
#confirmed_data_pox['confirmed'] = confirmed_data_pox.groupby(['Country'])['daily_cases'].cumsum()
#confirmed_data_pox = confirmed_data_pox.sort_values(by=['date'], ascending = True)

In [189]:
confirmed_data_pox.Country.unique()

array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil',
       'Bulgaria', 'Canada', 'Chile', 'Colombia', 'Croatia',
       'Czech Republic', 'Denmark', 'England', 'Finland', 'France',
       'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Hungary',
       'Iceland', 'Ireland', 'Israel', 'Italy', 'Latvia', 'Lebanon',
       'Luxembourg', 'Malta', 'Mexico', 'Morocco', 'Netherlands',
       'Northern Ireland', 'Norway', 'Peru', 'Poland', 'Portugal',
       'Romania', 'Scotland', 'Serbia', 'Singapore', 'Slovenia',
       'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland',
       'Taiwan', 'United Arab Emirates', 'United States', 'Venezuela',
       'Wales'], dtype=object)

In [190]:
len(confirmed_data_pox.Country.unique())

52

In [191]:
print(confirmed_data_pox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada'
 'Chile' 'Colombia' 'Croatia' 'Czech Republic' 'Denmark' 'England'
 'Finland' 'France' 'Georgia' 'Germany' 'Ghana' 'Gibraltar' 'Greece'
 'Hungary' 'Iceland' 'Ireland' 'Israel' 'Italy' 'Latvia' 'Lebanon'
 'Luxembourg' 'Malta' 'Mexico' 'Morocco' 'Netherlands' 'Northern Ireland'
 'Norway' 'Peru' 'Poland' 'Portugal' 'Romania' 'Scotland' 'Serbia'
 'Singapore' 'Slovenia' 'South Africa' 'South Korea' 'Spain' 'Sweden'
 'Switzerland' 'Taiwan' 'United Arab Emirates' 'United States' 'Venezuela'
 'Wales']


In [192]:
from datetime import date


def map_dateRange(df, w_countries):
    date_rng = pd.date_range(start='5/1/2022', end=confirmed_data_pox['date'].max())
    df_date = pd.DataFrame(date_rng,  columns=['date'])
    df = df[df['Country']==w_countries]
    df = df.merge(df_date, how='outer', on = ['date'])
    df['Country'] = df['Country'].fillna(w_countries)
    df['confirmed']=df['confirmed'].fillna(0)
    df = df.sort_values(by=['date'],  ascending=True, axis=0)
    return df

In [193]:
argentina = map_dateRange(confirmed_data_pox,'Argentina')
australia = map_dateRange(confirmed_data_pox,'Australia')
austria = map_dateRange(confirmed_data_pox,'Austria')
belgium = map_dateRange(confirmed_data_pox,'Belgium')
Bulgaria = map_dateRange(confirmed_data_pox,'Bulgaria')
brazil = map_dateRange(confirmed_data_pox,'Brazil')
canada = map_dateRange(confirmed_data_pox,'Canada')
czechrepublic = map_dateRange(confirmed_data_pox,'Czech Republic')
Chile =  map_dateRange(confirmed_data_pox, 'Chile')
Colombia =  map_dateRange(confirmed_data_pox, 'Colombia')
Croatia =  map_dateRange(confirmed_data_pox, 'Croatia')
Denmark = map_dateRange(confirmed_data_pox,'Denmark')
England = map_dateRange(confirmed_data_pox,'England')
Finland = map_dateRange(confirmed_data_pox,'Finland')
France = map_dateRange(confirmed_data_pox,'France')
Germany = map_dateRange(confirmed_data_pox,'Germany')
Georgia = map_dateRange(confirmed_data_pox,'Georgia')
Ghana = map_dateRange(confirmed_data_pox,'Ghana')
Gibraltar = map_dateRange(confirmed_data_pox,'Gibraltar')
Greece = map_dateRange(confirmed_data_pox,'Greece')
Hungary = map_dateRange(confirmed_data_pox,'Hungary')
Iceland = map_dateRange(confirmed_data_pox,'Iceland')
Ireland = map_dateRange(confirmed_data_pox,'Ireland')
Israel = map_dateRange(confirmed_data_pox,'Israel')
Italy = map_dateRange(confirmed_data_pox,'Italy')
Latvia = map_dateRange(confirmed_data_pox,'Latvia')
Lebanon = map_dateRange(confirmed_data_pox,'Lebanon')
Luxembourg = map_dateRange(confirmed_data_pox,'Luxembourg')
Mexico = map_dateRange(confirmed_data_pox,'Mexico')
Malta = map_dateRange(confirmed_data_pox,'Malta')
Morocco = map_dateRange(confirmed_data_pox,'Morocco')
Netherlands = map_dateRange(confirmed_data_pox,'Netherlands')
NorthernIreland = map_dateRange(confirmed_data_pox,'Northern Ireland')
Norway = map_dateRange(confirmed_data_pox,'Norway')
Poland = map_dateRange(confirmed_data_pox,'Poland')
Portugal = map_dateRange(confirmed_data_pox,'Portugal')
Romania = map_dateRange(confirmed_data_pox,'Romania')
Scotland = map_dateRange(confirmed_data_pox,'Scotland')
Serbia = map_dateRange(confirmed_data_pox,'Serbia')
Singapore =map_dateRange(confirmed_data_pox,'Singapore')
Slovenia = map_dateRange(confirmed_data_pox,'Slovenia')
Spain = map_dateRange(confirmed_data_pox,'Spain')
SouthAfrica = map_dateRange(confirmed_data_pox,'South Africa')
SouthKorea = map_dateRange(confirmed_data_pox,'South Korea')
Sweden = map_dateRange(confirmed_data_pox,'Sweden')
Switzerland = map_dateRange(confirmed_data_pox,'Switzerland')
Taiwan =  map_dateRange(confirmed_data_pox, 'Taiwan')
UnitedArabEmirates = map_dateRange(confirmed_data_pox,'United Arab Emirates')
UnitedStates = map_dateRange(confirmed_data_pox,'United States')
Venezuela = map_dateRange(confirmed_data_pox,'Venezuela')
Wales = map_dateRange(confirmed_data_pox,'Wales')







In [194]:
df_monkeypox = pd.concat([argentina, australia, austria, 
                          belgium, Bulgaria, brazil, canada, czechrepublic, Colombia,
                          Denmark, England, Finland, France, Germany, 
                          Ghana, Gibraltar, Greece, Hungary, Ireland, Israel, 
                          Italy, Latvia, Mexico, Malta, Morocco, Netherlands, 
                          NorthernIreland, Norway, Poland, Portugal, Scotland, Singapore, Slovenia, 
                          Spain, SouthKorea, SouthAfrica, Sweden, Switzerland,  Taiwan, UnitedArabEmirates, 
                          UnitedStates, Wales, Georgia, Croatia, Chile, Iceland, Lebanon, Luxembourg, Venezuela, Romania, Serbia], axis=0, ignore_index=True)


In [195]:
df_monkeypox['acc_confirmed'] = df_monkeypox.groupby(['Country'])['confirmed'].cumsum()
df_monkeypox['acc_confirmed'] = pd.to_numeric(df_monkeypox['acc_confirmed'], downcast='integer', errors='coerce')
df_monkeypox['confirmed'] = pd.to_numeric(df_monkeypox['confirmed'], downcast='integer', errors='coerce')


In [196]:
df_world = df_monkeypox.groupby(['date']).agg(acc_confirmed_sum=('acc_confirmed', 'sum'), confirmed_sum=('confirmed', 'sum')).reset_index()
df_world['World'] = 'World'

In [197]:
df_world = df_world[['World'] + ['date', 'acc_confirmed_sum', 'confirmed_sum']]
df_world['confirmed_sum'] = pd.to_numeric(df_world['confirmed_sum'], downcast='integer', errors='coerce')
df_world = df_world[['World', 'date', 'confirmed_sum'] + ['acc_confirmed_sum']]
df_world = df_world.rename(columns={'confirmed_sum': 'confirmed', 'acc_confirmed_sum': 'acc_confirmed', 'World': 'Country'})
df_monkeypox = pd.concat([df_world, df_monkeypox], axis=0, ignore_index=True)
df_monkeypox

,Country,date,confirmed,acc_confirmed
0,World,2022-05-01,0,0
1,World,2022-05-02,0,0
2,World,2022-05-03,0,0
3,World,2022-05-04,0,0
4,World,2022-05-05,0,0
...,...,...,...,...
3011,Serbia,2022-06-23,0,1
3012,Serbia,2022-06-24,0,1
3013,Serbia,2022-06-25,0,1
3014,Serbia,2022-06-26,0,1


In [198]:
#get world count
world_count = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]
world_count

4377

In [199]:
#set update date

df_monkeypox['date'].iloc[-1]

Timestamp('2022-06-27 00:00:00')

In [200]:
df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]-df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-2]



218

In [201]:
geo = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')


In [202]:
df_monkeypox_uk = df_monkeypox.loc[df_monkeypox['Country'].isin(['England', 'Northern Ireland', 'Scotland', 'Wales'])]


In [203]:
df_monkeypox_word = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]


In [204]:
df_monkeypox = df_monkeypox.loc[~(df_monkeypox['Country'].isin(['World', 'England', 'Northern Ireland', 'Scotland', 'Wales']))]


In [205]:
df_monkeypox_uk['latitude'] = 55.378051
df_monkeypox_uk['longitude'] = -3.435973

In [206]:
df_monkeypox = pd.merge(df_monkeypox, geo[['country', 'latitude', 'longitude']], how='inner', left_on=['Country'], right_on=['country'])
df_monkeypox = df_monkeypox.reset_index()


In [207]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_uk], axis=0, ignore_index=True)
df_monkeypox = df_monkeypox.drop(columns=['index', 'country'])


In [208]:
df_monkeypox_word['latitude']=0
df_monkeypox_word['longitude']=0

In [209]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_word], axis=0, ignore_index=True)

In [210]:
df_monkeypox = df_monkeypox.rename(columns={'confirmed': 'new_daily_cases'})

In [211]:
df_monkeypox.to_csv('df_monkeyPoxTimeSerie.csv', index_label=False)
df_monkeypox.to_csv('df_monkeyPoxTimeSerie2.csv')

In [212]:
#today world cases
df_monkeypox[df_monkeypox['Country']=='World']['acc_confirmed'].max()

4377

In [213]:
df_monkeypox
print(df_monkeypox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Bulgaria' 'Brazil' 'Canada'
 'Czech Republic' 'Colombia' 'Denmark' 'Finland' 'France' 'Germany'
 'Ghana' 'Gibraltar' 'Greece' 'Hungary' 'Ireland' 'Israel' 'Italy'
 'Latvia' 'Mexico' 'Malta' 'Morocco' 'Netherlands' 'Norway' 'Poland'
 'Portugal' 'Singapore' 'Slovenia' 'Spain' 'South Korea' 'South Africa'
 'Sweden' 'Switzerland' 'Taiwan' 'United Arab Emirates' 'United States'
 'Georgia' 'Croatia' 'Chile' 'Iceland' 'Lebanon' 'Luxembourg' 'Venezuela'
 'Romania' 'Serbia' 'England' 'Northern Ireland' 'Scotland' 'Wales'
 'World']


In [214]:
len(df_monkeypox['Country'].unique())

52

In [215]:
f"{world_count:,.0f}"

'4,377'

In [216]:
print(df_monkeypox[df_monkeypox['Country']=='World']['new_daily_cases'].iloc[-1]) 
print(df_monkeypox[df_monkeypox['Country']=='World']['new_daily_cases'].iloc[-2])

218
1


In [217]:
df_monkeypox.columns

Index(['Country', 'date', 'new_daily_cases', 'acc_confirmed', 'latitude',
       'longitude'],
      dtype='object')

In [218]:
# set rolling av



In [219]:
#df_monkeypox[df_monkeypox['Country']=='Argentina']

In [220]:
#set data for map

#df_monkeypox.groupby(['Country', 'latitude', 'longitude'])['acc_confirmed'].max().reset_index()

In [221]:
# Create dictionary of list
#testlist = df_monkeypox[['Country', 'latitude', 'longitude']]
#dict_of_locations = testlist.set_index('Country')[['latitude', 'longitude']].T.to_dict('dict')

In [222]:
#dict_of_locations